In [77]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import qutip as qt
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import importlib

%matplotlib ipympl

importlib.reload(sq_ext)
importlib.reload(sq)
np.set_printoptions(linewidth=500, formatter={'float': '{:.1e}'.format})

In [19]:
# Fundamental constants
h    = 6.626e-34
GHz = 1e9
e0   = 1.602e-19
Φ_0 = h/(2*e0)

# Single qubit

This notebook studies the qubit proposed by KIT. It's lumped element model is:
<img src='Other/Single qubit 4 nodes CC coupled to ground.jpg' width='500'>


# Circuit definitions
## Qubit

In [20]:
# Circuit parameters
Csh = 15
C   = 15
Cg  = 10
CJ  = 3
Lq  = 25
Lr  = 10
Δ   = 0.1
EJ  = 10.0
l = Lq*(Lq+4*Lr) - 4*Δ**2

# Initialize loop(s)
loop = sq.Loop(0.5)

# Circuit components
C_01 = sq.Capacitor(C,       'fF')
C_02 = sq.Capacitor(C,       'fF')
C_12 = sq.Capacitor(CJ+Csh,  'fF')
L_03 = sq.Inductor(Lr,       'nH')
L_31 = sq.Inductor(Lq/2 - Δ, 'nH',  loops=[loop])
L_23 = sq.Inductor(Lq/2 + Δ, 'nH',  loops=[loop])
JJ_12= sq.Junction(EJ,       'GHz', loops=[loop])

elements = {
    # Without ground node
    (0, 3): [L_03],
    (0, 1): [C_01],
    (0, 2): [C_02],
    (3, 1): [L_31],
    (1, 2): [C_12, JJ_12],
    (2, 3): [L_23],
}
# Create the circuits
qubit = sq.Circuit(elements)
qubit.description()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [21]:
sq_ext.print_flux_transformation(qubit)

Φ_1 = [ 0.00  0.00  1.00]
Φ_2 = [ 1.00  0.98  0.61]
Φ_3 = [-0.93  1.00  0.02]


# Equivalent circuits

### Equivalent circuit constructed as a fluxonium + resonator without coupling

In [22]:
loop_eq_fluxonium = sq.Loop(0.5)
loop_eq_ficticious = sq.Loop(0.5)

C_r_eq = sq.Capacitor(C / 2, 'fF')
L_r_eq = sq.Inductor(l / Lq, 'nH')

C_f_eq = sq.Capacitor(C / 2 + Csh + CJ, 'fF')
L_f_eq = sq.Inductor(l / (Lq + 4 * Lr), 'nH', loops=[loop_eq_ficticious])
JJ_f_eq = sq.Junction(EJ, 'GHz', loops=[loop_eq_fluxonium])

equiv_elements = {
    (0, 1): [C_r_eq, L_r_eq, C_f_eq, L_f_eq, JJ_f_eq],
}
qubit_unc = sq.Circuit(equiv_elements)
qubit_unc.description()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Equivalent circuit constructed as a fluxonium + resonator + coupling

In [25]:
loop_eq_fluxonium = sq.Loop(0.5)
loop_eq_ficticious = sq.Loop(0.5)

C_r_eq = sq.Capacitor(C / 2, 'fF')
L_r_eq = sq.Inductor(1 / (1 / (l / Lq) - 1 / (l / Δ)), 'nH')

C_f_eq = sq.Capacitor(C / 2 + Csh + CJ, 'fF')
L_f_eq = sq.Inductor(1 / (1 / (l / (Lq + 4 * Lr)) - 1 / (l / Δ)), 'nH', loops=[loop_eq_fluxonium, loop_eq_ficticious])
JJ_f_eq = sq.Junction(EJ, 'GHz', loops=[loop_eq_fluxonium])

L_c_eq = sq.Inductor(l / Δ, 'nH')

equiv_elements = {
    (0, 1): [C_r_eq, L_r_eq],
    (1, 2): [L_c_eq],
    (0, 2): [C_f_eq, L_f_eq, JJ_f_eq],
}
qubit_equiv = sq.Circuit(equiv_elements)
qubit_equiv.description()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [26]:
sq_ext.print_flux_transformation(qubit_equiv)

Φ_1 = [-1.00  0.00]
Φ_2 = [ 0.02  1.00]


### Resonator

In [27]:
resonator_elements = {
    (0, 1): [sq.Capacitor(C/2, 'fF'), sq.Inductor (l/Lq, 'nH')],
}
resonator = sq.Circuit(resonator_elements )
resonator.description()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Fluxonium

In [29]:
loop_fluxonium = sq.Loop(0.5)
fluxonium_elements = {
    (0, 1): [sq.Capacitor(C/2 + Csh + CJ, 'fF'), sq.Inductor(l/(Lq+4*Lr), 'nH', loops=[loop_fluxonium]), sq.Junction(EJ, 'GHz', loops=[loop_fluxonium])],
    # (0, 1): [sq.Inductor(l/(Lq+4*Lr), 'nH', loops=[loop_fluxonium]), sq.Junction(EJ, 'GHz', loops=[loop_fluxonium], cap=sq.Capacitor(C/2 + Csh + CJ, 'fF'))],
}
fluxonium = sq.Circuit(fluxonium_elements)
fluxonium.description()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Spectrum and eigenstates

## Resonator spectrum

In [31]:
L_r = l/Lq * 1e-9
C_r = C/2 * 1e-15
w_r = 1/np.sqrt(L_r*C_r) / 2 / np.pi / 1e9
E_harm = w_r * np.arange(0,8)
E_harm

array([0.0e+00, 7.2e+00, 1.4e+01, 2.2e+01, 2.9e+01, 3.6e+01, 4.3e+01, 5.0e+01])

## Potential energy and eigenstates resoantor

In [32]:
E_L_r = (Φ_0/(2*np.pi))**2/L_r / h

φ_r = np.linspace(-3,3,200)
n_eig = 2

V_r = 0.5 * E_L_r * (φ_r)**2

resonator.set_trunc_nums([20])
_, _ = resonator.diag(n_eig)

states_r = [resonator.eig_phase_coord(k=i, grid=[φ_r/2/np.pi]) for i in range(n_eig)]

fig, ax = plt.subplots()
ax.plot(φ_r, (V_r-V_r.min())/1e9, 'k:' )
[ax.plot(φ_r, np.abs(state)**2) for state in states_r]
ax.set_ylim([-.1,3])
fig.show()

## Potential energy and eigenstates of the fluxonium


In [33]:
L_q = l / (Lq + 4*Lr) * 1e-9
E_J = EJ * 1e9

In [34]:
E_L_f = (Φ_0/(2*np.pi))**2/L_q / h
E_C = e0**2 / (2*(C/2 + Csh + CJ)*1e-15) / h

In [35]:
print(E_L_f/GHz, E_C/GHz, E_J/GHz)

6.539994179227587 0.7594573959979404 10.0


In [36]:
φ_q = np.linspace(-3,3,200)
φ_ext = np.pi
# V = 0.5 * 1/L_q * (Φ_q)**2 - E_J * np.cos((Φ_q-Φ_ext)*2*np.pi)
V = 0.5 * E_L_f * (φ_q)**2 - E_J * np.cos(φ_q-φ_ext)

n_eig = 2
fluxonium.set_trunc_nums([20])
loop_fluxonium.set_flux(φ_ext/2/np.pi)
_, _ = fluxonium.diag(n_eig)
states_f = [fluxonium.eig_phase_coord(k=i, grid=[φ_q/2/np.pi]) for i in range(n_eig)]


fig, ax = plt.subplots()
ax.plot(φ_q, (V-V.min())/1e9, 'k:' )

[ax.plot(φ_q, np.abs(state)**2) for state in states_f]
# ax.vlines(φ_min,-.1,3)
ax.set_ylim([-.1,3])
fig.show()

## Spectrum vs external flux

In [100]:
qubit.set_trunc_nums([1, 15, 15])
fluxonium.set_trunc_nums([10])

# spectrum of the qubit
# phi = np.linspace(0,1,100)
phi = np.linspace(0.36,0.38,500)
n_eig=8
spec = np.zeros((n_eig, len(phi)))
E_qubit = np.zeros((n_eig, len(phi)))

eig_colors = plt.get_cmap('viridis_r')(np.linspace(0, 255, n_eig).astype('int'))
for i in range(len(phi)):
    # set the external flux for the loop
    loop_fluxonium.set_flux(phi[i])
    loop.set_flux(phi[i])
    # diagonalize the qubit
    spec[:, i] = qubit.diag(n_eig)[0]
    E_qubit[:, i] = fluxonium.diag(n_eig)[0]

fig, ax = plt.subplots(dpi=150)
for i in range(n_eig):
    ax.plot(phi, spec[i,:]- spec[0,:], color=eig_colors[i], linewidth=1)
for i in range(n_eig//2):
    ax.plot([phi[0],phi[-1]], [E_harm[i],E_harm[i]], ':k' )
    ax.plot(phi, E_qubit[i,:]- E_qubit[0,:], color='r', linestyle = ':')

ax.set_xlabel(r"$\Phi_{ext}/\Phi_0$", fontsize=13)
ax.set_ylabel(r"$f_i-f_0$[GHz]", fontsize=13)
fig.suptitle('(a)')
ax.set_ylim(7,7.5)
fig.show()
figs.export('Full_spectrum.pdf')

## Eigenstates of the qubit in phase base

In [38]:
qubit.set_trunc_nums([1,15,15])

# creat the phase grid
phi1 = 0
phi2 = np.linspace(-0.005, 0.005, 100)
phi3 = np.linspace(-0.5, 0.5  , 100)
grid = [phi1, phi2, phi3]

state0_vs_ext_flux = []
state1_vs_ext_flux = []
state2_vs_ext_flux = []
state3_vs_ext_flux = []

ext_flux_list = [0.5, 0.55, 0.65, 1]
# loop thorugh several values of external flux
for ext_flux in ext_flux_list:
    loop.set_flux(ext_flux)
    _, _ = qubit.diag(n_eig=5)

    # Calculate eigenstates in phase coord
    state0_vs_ext_flux.append(qubit.eig_phase_coord(0, grid=grid))
    state1_vs_ext_flux.append(qubit.eig_phase_coord(1, grid=grid))
    state2_vs_ext_flux.append(qubit.eig_phase_coord(2, grid=grid))
    state3_vs_ext_flux.append(qubit.eig_phase_coord(3, grid=grid))

In [39]:
fig, axs = plt.subplots(4, 4, figsize=(4*4, 4*4), sharey='row', sharex = 'col', dpi=150)

for i, ext_flux in enumerate(ext_flux_list):
    axs[i, 0].contourf(phi3, phi2, np.abs(state0_vs_ext_flux[i].T) ** 2, cmap="Blues", )
    axs[i, 1].contourf(phi3, phi2, np.abs(state1_vs_ext_flux[i].T) ** 2, cmap="Greens" )
    axs[i, 2].contourf(phi3, phi2, np.abs(state2_vs_ext_flux[i].T) ** 2, cmap="Oranges")
    axs[i, 3].contourf(phi3, phi2, np.abs(state3_vs_ext_flux[i].T) ** 2, cmap="Reds"   )
    for j in range(4):
        axs[-1, j].set_xlabel(r"$\varphi_2$", fontsize=13)
    axs[i, 0].set_ylabel(r"$\varphi_3$", fontsize=13)
    axs[i, 0].set_title('$\phi_{ext}/\phi_{0}$ ='+f'{ext_flux}', fontsize=13, y=0.9 )
fig.suptitle('Eigenstates 0, 1, 2 and 3 (from left to right) in phase base')
plt.subplots_adjust(wspace=0)
plt.subplots_adjust(hspace=0)
plt.show()

# Fluxonium-Resonator internal coupling in KIT's qubit

In [52]:
fluxonium.set_trunc_nums([15])
resonator.set_trunc_nums([15])
loop_fluxonium.set_flux(0.5)
loop.set_flux(0.5)
_ = qubit.diag(2)
_ = fluxonium.diag(2)
_ = resonator.diag(2)

In [53]:
Δ01 = (qubit.efreqs[1]-qubit.efreqs[0])
Δ01

1.9453602983437293

In [54]:
L_c = l / Δ * 1e-9
E_L_c = (Φ_0/(2*np.pi))**2/L_c / h / GHz
E_L_c

0.01006152950650398

In [55]:
Φ_f = sq_ext.decomposition_in_pauli_2x2(fluxonium.flux_op(0, basis='eig').__array__())[1]
Φ_f

38.41921243891211

In [56]:
Φ_r = sq_ext.decomposition_in_pauli_2x2(resonator.flux_op(0, basis='eig').__array__())[1]
Φ_r

38.365883888557

In [68]:
Φ_r * Φ_f

1473.987043521107

In [57]:
g = Φ_r * Φ_f / L_c / GHz
g

0.09070912782598382

In [58]:
g / Δ01 * 100

4.662844610492624

### Analytical approximation of the fluxonim-resonator coupling

In [59]:
dVdφ =  lambda φ: E_L_f*φ + E_J*np.sin(φ+np.pi)
φ_min = sp.optimize.fsolve(dVdφ, 1)[0]
φ_min

1.527628932582084

In [60]:
Φ_f_an =(Φ_0 / 2*np.pi) * φ_min
Φ_f_an

4.962456082226201e-15

In [61]:
Φ_r_an = np.sqrt(sq.hbar/2 * np.sqrt(L_r/C_r))
Φ_r_an

3.939883949812526e-16

In [62]:
g_an = Φ_f_an * Φ_r_an /L_c / h /GHz
g_an

0.18158752095082853

In [63]:
g_an / Δ01 * 100

9.334390195247188

# Alternative method to calculate the coupling

In [155]:
qubit.set_trunc_nums([1, 5, 10])
resonator.set_trunc_nums([5])
fluxonium.set_trunc_nums([10])

In [159]:
H = qubit.hamiltonian()
H_r = resonator.hamiltonian()
H_f = fluxonium.hamiltonian()
H

Quantum object: dims = [[5, 10], [5, 10]], shape = (50, 50), type = oper, isherm = True
Qobj data =
[[4.9e+10 0.0e+00 -1.7e+10 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 6.5e+10 0.0e+00 ... -2.1e+00 0.0e+00 1.7e-01]
 [-1.7e+10 0.0e+00 8.7e+10 ... 0.0e+00 -3.9e+00 0.0e+00]
 ...
 [0.0e+00 -2.1e+00 0.0e+00 ... 4.4e+11 0.0e+00 -4.2e+10]
 [0.0e+00 0.0e+00 -3.9e+00 ... 0.0e+00 4.5e+11 0.0e+00]
 [0.0e+00 1.7e-01 0.0e+00 ... -4.2e+10 0.0e+00 5.3e+11]]

In [160]:
H_u = qt.tensor(H_r, qt.qeye(fluxonium.m )) + qt.tensor(qt.qeye(resonator.m), H_f)
H_u

Quantum object: dims = [[5, 10], [5, 10]], shape = (50, 50), type = oper, isherm = True
Qobj data =
[[4.9e+10 0.0e+00 -1.7e+10 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 6.5e+10 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [-1.7e+10 0.0e+00 8.7e+10 ... 0.0e+00 0.0e+00 0.0e+00]
 ...
 [0.0e+00 0.0e+00 0.0e+00 ... 4.4e+11 0.0e+00 -4.2e+10]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 4.5e+11 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... -4.2e+10 0.0e+00 5.3e+11]]

In [183]:
H_c = H.transform(H_u.eigenstates()[1]) - H_u
H_c

Quantum object: dims = [[5, 10], [5, 10]], shape = (50, 50), type = oper, isherm = True
Qobj data =
[[-7.2e+09 0.0e+00 1.7e+10 ... 6.0e-01 -5.4e+01 0.0e+00]
 [0.0e+00 -1.1e+10 0.0e+00 ... 0.0e+00 0.0e+00 8.3e-01]
 [1.7e+10 0.0e+00 -5.9e+09 ... -3.3e+00 -5.6e+02 0.0e+00]
 ...
 [6.0e-01 0.0e+00 -3.3e+00 ... 2.7e+10 -7.9e+08 4.2e+10]
 [-5.4e+01 0.0e+00 -5.6e+02 ... -7.9e+08 5.5e+10 0.0e+00]
 [0.0e+00 8.3e-01 0.0e+00 ... 4.2e+10 0.0e+00 2.1e+10]]

In [235]:
eigs_H_u = np.array( [ eig.__array__()[:,0] for eig in H_u.eigenstates()[1] ] ) 
eigs_H_c = np.array( [ eig.__array__()[:,0] for eig in H_c.eigenstates()[1] ] )


In [238]:
sq_ext.H_eff_SWT_eigs(eigs_H_u, eigs_H_c, H_u.eigenenergies())

array([[ 1.64015940e+11+0.j,  2.52574204e+09+0.j, -4.53846984e+10+0.j, ...,  2.23568213e+06+0.j, -3.45352578e+07+0.j, -1.57727046e+07+0.j],
       [ 2.52574204e+09+0.j,  2.05411262e+11+0.j,  1.18271095e+09+0.j, ..., -3.32548658e+07+0.j, -5.32696592e+05+0.j,  4.16558641e+07+0.j],
       [-4.53846984e+10+0.j,  1.18271095e+09+0.j,  2.34211510e+11+0.j, ..., -9.10821563e+07+0.j,  1.10619557e+08+0.j,  5.12603011e+07+0.j],
       ...,
       [ 2.23568213e+06+0.j, -3.32548658e+07+0.j, -9.10821563e+07+0.j, ...,  2.95973897e+11+0.j,  3.92431775e+09+0.j, -3.28691960e+10+0.j],
       [-3.45352578e+07+0.j, -5.32696592e+05+0.j,  1.10619557e+08+0.j, ...,  3.92431775e+09+0.j,  3.66482801e+11+0.j, -9.16278169e+09+0.j],
       [-1.57727046e+07+0.j,  4.16558641e+07+0.j,  5.12603011e+07+0.j, ..., -3.28691960e+10+0.j, -9.16278169e+09+0.j,  3.75972566e+11+0.j]])